In [1]:
from datasets import load_from_disk
from datasets.filesystems import S3FileSystem

In [2]:
from omegaconf import OmegaConf
params = OmegaConf.load('../configs/predict_generator_config.yaml')

In [3]:
params.input

'dmitrii/results/retriever_predictions'

In [4]:
f's3://{params.bucket_name}/{params.input}'

's3://tcr-internal/dmitrii/results/retriever_predictions'

In [5]:
s3 = S3FileSystem()
dataset = load_from_disk(f's3://{params.bucket_name}/{params.input}', storage_options=s3.storage_options)

/Users/dmitriimaksimov/Desktop/GitHub/master_thesis_RAG_LLM/.venv/lib/python3.10/site-packages/fsspec/spec.py:79: FutureWarning: S3FileSystem is deprecated and will be removed in the next major version of datasets. Use s3fs.S3FileSystem instead.
  obj = super().__call__(*args, **kwargs)
/Users/dmitriimaksimov/Desktop/GitHub/master_thesis_RAG_LLM/.venv/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [6]:
my_dataset = load_from_disk('../results/retriever_predictions/')

In [14]:
import sys
sys.path.append('../')
from src.evaluation.evaluation import calculate_HitRate, calculate_DocHitRate

In [25]:
hit_rate = calculate_HitRate(dataset['val'])

In [26]:
hit_rate

0.7049180327868853

In [16]:
hit_rate = calculate_HitRate(my_dataset['train'])

In [20]:
hit_rate

0.9142656359803233

In [9]:
my_dataset['train'][14]

{'question': 'Welchen Durchmesser hat die Abschirmung der USB-Kabeltypen gemäß der angegebenen Spezifikationen?',
 'ground_truth': '3,20 mm und 4,10 mm',
 'page': 75,
 'document': 'leoni_fahrzeugleitungen',
 'contexts': ['57 Fahrzeugleitungen mehradrig LEONI   Adascar ® www.leoni-automotive-cables.com\nAdvanced Automotive Special Cables.\nAderdurchmesser nom. Manteldurchmesser nom. Abschirmung Adermaterial Mantelmaterial Leiterwiderstand   max. Betriebstemperatur\n[mm] [mm] [ Ω /km] [°C]\n2,10 7,30 – PVC PVC 37,1 –40 bis +105\n1,80 4,80 – PVC PVC 24,7 –40 bis +125\n1,25 6,30 – PVC TPE-U 55,5 –40 bis +105\n1,50 6,30 – ETFE TPE-U 38,2 –40 bis +125\n2,20 6,20 – TPE-U TPE-U 25,4 –40 bis +125\n1,40 4,20 – TPE-S TPE-O 54,4 –40 bis +90\n1,55 5,50 – TPE-O TPE-U 37,1 –40 bis +125\n1,05 6,30 – ETFE TPE-U 86,5 –40 bis +125\n1,00 3,70 – TPE-E TPE-U 86,5 –40 bis +125\n1,10 4,50 – TPE-E TPE-U 86,5 –30 bis +85\n1,14 3,80 – TPE-E TPE-U 72,9 –40 bis +105\n2,80 7,00 C PVC PVC 7,6 –40 bis +105\n2,00 8,00

In [10]:
dataset['train'][14]

{'question': 'Welchen Durchmesser hat die Abschirmung der USB-Kabeltypen gemäß der angegebenen Spezifikationen?',
 'ground_truth': '3,20 mm und 4,10 mm',
 'page': 75,
 'document': 'leoni_fahrzeugleitungen',
 'contexts': ['57 Fahrzeugleitungen mehradrig LEONI   Adascar ® www.leoni-automotive-cables.com\nAdvanced Automotive Special Cables.\nAderdurchmesser nom. Manteldurchmesser nom. Abschirmung Adermaterial Mantelmaterial Leiterwiderstand   max. Betriebstemperatur\n[mm] [mm] [ Ω /km] [°C]\n2,10 7,30 – PVC PVC 37,1 –40 bis +105\n1,80 4,80 – PVC PVC 24,7 –40 bis +125\n1,25 6,30 – PVC TPE-U 55,5 –40 bis +105\n1,50 6,30 – ETFE TPE-U 38,2 –40 bis +125\n2,20 6,20 – TPE-U TPE-U 25,4 –40 bis +125\n1,40 4,20 – TPE-S TPE-O 54,4 –40 bis +90\n1,55 5,50 – TPE-O TPE-U 37,1 –40 bis +125\n1,05 6,30 – ETFE TPE-U 86,5 –40 bis +125\n1,00 3,70 – TPE-E TPE-U 86,5 –40 bis +125\n1,10 4,50 – TPE-E TPE-U 86,5 –30 bis +85\n1,14 3,80 – TPE-E TPE-U 72,9 –40 bis +105\n2,80 7,00 C PVC PVC 7,6 –40 bis +105\n2,00 8,00

In [35]:
dataset['train']

Dataset({
    features: ['question', 'ground_truth', 'page', 'document', 'contexts', 'distances', 'contexts_pages', 'contexts_documents'],
    num_rows: 1423
})

In [36]:
def ds_preprocessing(row):
    for i, doc in enumerate(row['contexts_documents']):
        if doc == row['document'] and row['page'] == row['contexts_pages'][i]:
            row['ground_truth'] = row['ground_truth']
            return row
    row['ground_truth'] = 'Leider kann ich diese Frage nicht einmal mit den bereitgestellten Informationen beantworten'
    return row
    

In [38]:
dataset = dataset.map(ds_preprocessing)

In [40]:
dataset['train']['ground_truth']

['Überwachung und Qualitätssicherung der Crimp-Kontaktierung',
 'TPE-U oder TPE-A/TPE-E',
 'Länge des Drahtcrimps (100 %)',
 'Stellen Sie die Höhe der Presse ein.',
 'ISO 6722-1',
 'LEONI Adascar® Truck 10504 mit 84 Einzeldrähten.',
 'PVC',
 'Sie dienen zur Bestimmung des Durchgangswiderstands der Verkabelung.',
 'Der Abstand von den äußeren Enden der Crimpflanken bis zum Ende der Crimpzunge.',
 'Die Breite des Crimps, die mit einem Messgerät bestimmt werden kann.',
 'Die Anpassung an die Eigenschaften individueller vorhandener Kontakte.',
 'Die Streuung in Prozent ist die Standardabweichung relativ zum arithmetischen Mittelwert der Gut-Crimps, ausgedrückt in Prozent.',
 '1,5 Stunden',
 'Die Antwort ist nicht im gegebenen Text enthalten, da der Abschnitt 5.1 nicht gezeigt wird.',
 '3,20 mm und 4,10 mm',
 'min. 99,85%',
 'Der Leiterdurchmesser von Control 87010 ist 0,19 mm und von Control 87020 ist 0,26 mm.',
 "Werkzeugmacher-Mikroskop (Toolmaker's Microscope).",
 'Abschmelzungen in der

In [31]:
dataset['train']['page'][:2]

[8, 69]

In [33]:
dataset['train']['contexts_documents'][:2]

[['GS 95006-7-5 Kfz-Kontaktierungen Überwachungskriterien von Crimp-1',
  'GS 95006-7-5 Kfz-Kontaktierungen Überwachungskriterien von Crimp-1',
  'GS 95006-7-5 Kfz-Kontaktierungen Überwachungskriterien von Crimp-1'],
 ['leoni_fahrzeugleitungen',
  'leoni_fahrzeugleitungen',
  'leoni_fahrzeugleitungen']]

In [34]:
dataset['train']['contexts_pages'][:2]

[[8, 9, 14], [69, 52, 51]]

In [1]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained('LeoLM/leo-mistral-hessianai-7b-chat')
# model = AutoModelForCausalLM.from_pretrained('LeoLM/leo-mistral-hessianai-7b-chat')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
tokenizer.pad_token_id

2

In [5]:
tokenizer.eos_token_id

2

In [3]:
import torch
model.eval()
with torch.no_grad():
    res = model(**tokenizer(['I am Dima'], return_tensors='pt'))

In [5]:
res['logits'].shape

torch.Size([1, 5, 32128])

In [6]:
res['logits'].argmax(-1)

tensor([[32000, 28716,  5489, 24418, 28725]])

In [10]:
tokenizer.decode([32000, 28716,  5489, 24418, 28725], skip_special_tokens=False)

'<|im_start|>h currentlyharm,'

In [11]:
tokenizer(['I am Dima'], return_tensors='pt')

{'input_ids': tensor([[   1,  315,  837,  384, 4250]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [13]:
tokenizer.convert_ids_to_tokens(28725)

','

In [9]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>',
 'additional_special_tokens': ['<unk>',
  '<s>',
  '</s>',
  '<|im_start|>',
  '<|im_end|>']}

In [ ]:
mode

In [4]:
data_collator = DataCollatorForCompletionOnlyLM('-#', tokenizer=tokenizer, mlm=False)

In [10]:
text = ["I am Dima-#WTF"]

# Tokenize the text
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)


# Use the data collator on the tokenized text
collated_inputs = data_collator([inputs])

# Print the output
print(collated_inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [6]:
inputs['input_ids']

tensor([[    1,   315,   837,   384,  4250, 28733, 28771, 28780, 12352],
        [    2,     2,     2,     2,     2,     2,     1,   330,  5309]])

In [15]:
tokenizer.convert_ids_to_tokens(12352)

'TF'

In [15]:
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
import torch

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("LeoLM/leo-mistral-hessianai-7b-chat")

# Prepare the text
text = ["I am Dima-#WTF", "asd"]

# Tokenize the text
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Initialize the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # False because GPT-2 is not a masked language model
)

# Use the data collator on the tokenized text
collated_inputs = data_collator([inputs])

# Print the output
print(collated_inputs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[[    1,   315,   837,   384,  4250, 28733, 28771, 28780, 12352],
         [    2,     2,     2,     2,     2,     2,     1,   390, 28715]]]), 'attention_mask': tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 1, 1, 1]]]), 'labels': tensor([[[    1,   315,   837,   384,  4250, 28733, 28771, 28780, 12352],
         [ -100,  -100,  -100,  -100,  -100,  -100,     1,   390, 28715]]])}


In [14]:
collated_inputs['input_ids']

tensor([[[    1,   315,   837,   384,  4250, 28733, 28771, 28780, 12352]]])

In [ ]:
SFTTrainer()

In [ ]:
from transformers.trainer_utils import EvalPrediction

In [15]:
# from aim.hugging_face import AimCallback
from aim import Run
aim_callback = Run(repo='aim://77.25.23.186:53800', experiment='your_experiment_name')

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:77.25.23.186:53800: Failed to connect to remote host: Connection refused"
	debug_error_string = "UNKNOWN:failed to connect to all addresses; last error: UNKNOWN: ipv4:77.25.23.186:53800: Failed to connect to remote host: Connection refused {created_time:"2024-01-07T20:10:54.296397+01:00", grpc_status:14}"
>

In [12]:
aim_callback.close

In [9]:
from omegaconf import OmegaConf
params = OmegaConf.load('../configs/train_generator_config.yaml')

In [10]:
import sys
sys.path.append("../") 

In [15]:
from peft import LoraConfig
from dataclasses import asdict
from src.entities.train_generator_params import read_train_generator_params

In [12]:
params = read_train_generator_params(params)

In [16]:
LoraConfig(**asdict(params.lora_config))

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=32, target_modules={'k_proj', 'gate_proj', 'o_proj', 'lm_head', 'q_proj', 'up_proj', 'v_proj', 'down_proj'}, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [18]:
b = False

In [21]:
a = not b

In [22]:
a

True

In [ ]:
from transformers import MistralForCausalLM